In [41]:
import robin_stocks as r
from datetime import datetime
from dateutil import tz
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'

from_zone = tz.tzutc()
to_zone = tz.tzlocal()
def utcToLocal(strDate):
    utc = datetime.strptime(strDate, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

import configparser
config = configparser.RawConfigParser()
configFilePath = '/Users/philipmassey/.tokens/robinhood.cfg'
config.read(configFilePath)
rhuser = config.get('login', 'user')
rhpwd = config.get('login', 'pwd')
login = r.login(rhuser,rhpwd)

def diffMaxHighLow(df, strike_price,volume):
    print('Strike\tVolume\tHigh\tLow\tChange\tPercent')
    high_price = max(df.high_price)
    low_price = min(df.high_price)
    diff = round((high_price - low_price),2)
    percent = round(diff/low_price,2)
    print('${:.2f}\t{:,.0f}\t{}\t{}\t{}\t{:,.0%}'.format(strike_price,volume,high_price,low_price,diff,percent))

def figForExpriation(strike_price,volume,optionType,span):
    df = pd.DataFrame(r.get_option_historicals(symbol,expirationDate,strike_price,optionType,span)['data_points'])
    df['strike_price'] = strike_price
    df[['strike_price','high_price','low_price']] = df[['strike_price','high_price','low_price']].apply(pd.to_numeric)
    df = df[df.high_price != df.high_price[0:1][0]]   #filter no changes form the starting date
    print()
    diffMaxHighLow(df,float(strike_price),float(volume))
    local = utcToLocal(df.iloc[-1:].begins_at.values[-1])
    title = '{} Expiration:{}, Time:{:{dfmt} {tfmt}}'.format(symbol,expirationDate,local,dfmt='%Y-%m-%d', tfmt='%H:%M')
    fig = px.line(df, x="begins_at", y="close_price", color="strike_price", line_group="strike_price", hover_name="strike_price",
            line_shape="spline", render_mode="svg")
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Option price",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        ))
    fig.show()

### Expiration Dates

In [33]:
symbol = 'SRC'
expiration_dates = r.get_chains(symbol)['expiration_dates']
print(expiration_dates)

['2020-06-19', '2020-07-17', '2020-10-16', '2020-11-20', '2021-01-15']


### Strike Prices for Expiration Date

In [34]:
expirationDate = expiration_dates[1]
volume_limit = 0
optionData = r.find_options_for_list_of_stocks_by_expiration_date([symbol], expirationDate=expirationDate,optionType='call') 
dfoptions = pd.DataFrame((filter(lambda x:x['volume']>volume_limit,optionData)))
#dfoptions[['strike_price','break_even_price','volume','last_trade_price', 'previous_close_price', 'last_trade_size','open_interest']].sort_values(by='strike_price')
#dfoptions[['strike_price','break_even_price','volume','last_trade_price', 'previous_close_price', 'last_trade_size','open_interest']].sort_values(by='volume',ascending=False)
# df1 = dfoptions[['strike_price','volume']][dfoptions.strike_price=='237.5000']
# dfstrike_prices = pd.concat([df1,dfstrike_prices])
dfstrike_prices = dfoptions.sort_values(by='volume',ascending=False)[0:5][['strike_price','volume']].apply(pd.to_numeric)
dfstrike_prices.sort_values(by='strike_price',ascending=True,inplace=True)
dfstrike_prices

*.

,strike_price,volume
2,30.0,21
1,33.0,20
0,34.0,134
5,37.0,39
6,40.0,40


In [44]:
optionType = 'call'
span = 'day'
idx = 4
strike_price = dfstrike_prices.iloc[idx].strike_price
volume = dfstrike_prices.iloc[idx].volume
#strike_price = 40.0
figForExpriation(strike_price,volume,optionType,span)

*.
Strike	Volume	High	Low	Change	Percent
$40.00	40	1.23	0.43	0.8	186%


In [45]:
optionType = 'call'
span = 'week'
figForExpriation(strike_price,volume,optionType,span)


*.
Strike	Volume	High	Low	Change	Percent
$40.00	40	1.23	0.28	0.95	339%


In [46]:
span = 'year'
figForExpriation(strike_price,volume,optionType,span)

*.
Strike	Volume	High	Low	Change	Percent
$40.00	40	15.25	0.01	15.24	152,400%
